In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
!ls

AITSdigit2.ipynb  AITSdigit.ipynb  test.csv  train.csv


In [0]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [0]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [23]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = X_train.as_matrix().reshape(33600, 784) #(33600, 784)
X_cv = X_cv.as_matrix().reshape(8400, 784) #(8400, 784)

X_test = X_test.as_matrix().reshape(28000, 784)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [25]:
print((min(X_train[1]), max(X_train[1])))

(0, 255)


In [26]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7



[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [0]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)



In [28]:

# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
_________________________________________________________________


In [0]:

# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

In [30]:


# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

history1 = model.fit(X_train, y_train, batch_size = batch_size, epochs = training_epochs,verbose = 2, validation_data=(X_cv, y_cv))

W0806 14:45:01.227365 139844299061120 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0806 14:45:01.303502 139844299061120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 4s - loss: 1.8540 - acc: 0.4985 - val_loss: 1.0044 - val_acc: 0.7600
Epoch 2/20
 - 3s - loss: 0.6479 - acc: 0.8295 - val_loss: 0.4634 - val_acc: 0.8730
Epoch 3/20
 - 3s - loss: 0.4090 - acc: 0.8836 - val_loss: 0.3615 - val_acc: 0.8976
Epoch 4/20
 - 3s - loss: 0.3373 - acc: 0.9025 - val_loss: 0.3121 - val_acc: 0.9101
Epoch 5/20
 - 3s - loss: 0.2979 - acc: 0.9138 - val_loss: 0.2891 - val_acc: 0.9170
Epoch 6/20
 - 3s - loss: 0.2683 - acc: 0.9227 - val_loss: 0.2650 - val_acc: 0.9239
Epoch 7/20
 - 3s - loss: 0.2453 - acc: 0.9298 - val_loss: 0.2556 - val_acc: 0.9256
Epoch 8/20
 - 3s - loss: 0.2272 - acc: 0.9351 - val_loss: 0.2321 - val_acc: 0.9338
Epoch 9/20
 - 3s - loss: 0.2101 - acc: 0.9378 - val_loss: 0.2175 - val_acc: 0.9364
Epoch 10/20
 - 3s - loss: 0.1952 - acc: 0.9437 - val_loss: 0.2053 - val_acc: 0.9396
Epoch 11/20
 - 3s - loss: 0.1829 - acc: 0.9468 - val_loss: 0.1954 - val_acc: 0.9427
Epoch 12/20
 - 3s - loss: 0.1708 - a

In [31]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 4s - loss: 0.3424 - acc: 0.8977 - val_loss: 0.1590 - val_acc: 0.9512
Epoch 2/20
 - 4s - loss: 0.1265 - acc: 0.9605 - val_loss: 0.1166 - val_acc: 0.9651
Epoch 3/20
 - 4s - loss: 0.0836 - acc: 0.9744 - val_loss: 0.0987 - val_acc: 0.9706
Epoch 4/20
 - 4s - loss: 0.0603 - acc: 0.9806 - val_loss: 0.1119 - val_acc: 0.9657
Epoch 5/20
 - 4s - loss: 0.0452 - acc: 0.9851 - val_loss: 0.1021 - val_acc: 0.9725
Epoch 6/20
 - 4s - loss: 0.0376 - acc: 0.9874 - val_loss: 0.0992 - val_acc: 0.9755
Epoch 7/20
 - 4s - loss: 0.0363 - acc: 0.9874 - val_loss: 0.0954 - val_acc: 0.9735
Epoch 8/20
 - 4s - loss: 0.0274 - acc: 0.9913 - val_loss: 0.1027 - val_acc: 0.9750
Epoch 9/20
 - 4s - loss: 0.0180 - acc: 0.9940 - val_loss: 0.1055 - val_acc: 0.9748
Epoch 10/20
 - 4s - loss: 0.0202 - acc: 0.9935 - val_loss: 0.1166 - val_acc: 0.9726
Epoch 11/20
 - 4s - loss: 0.0233 - acc: 0.9926 - val_loss: 0.1065 - val_acc: 0.9765
Epoch 12/20
 - 4s - loss: 0.0178 - a

In [32]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 5s - loss: 0.3404 - acc: 0.8964 - val_loss: 0.1994 - val_acc: 0.9412
Epoch 2/20
 - 4s - loss: 0.1265 - acc: 0.9610 - val_loss: 0.1289 - val_acc: 0.9611
Epoch 3/20
 - 4s - loss: 0.0828 - acc: 0.9747 - val_loss: 0.1030 - val_acc: 0.9690
Epoch 4/20
 - 4s - loss: 0.0602 - acc: 0.9809 - val_loss: 0.0933 - val_acc: 0.9719
Epoch 5/20
 - 4s - loss: 0.0471 - acc: 0.9851 - val_loss: 0.1010 - val_acc: 0.9721
Epoch 6/20
 - 4s - loss: 0.0378 - acc: 0.9871 - val_loss: 0.1171 - val_acc: 0.9690
Epoch 7/20
 - 4s - loss: 0.0324 - acc: 0.9890 - val_loss: 0.1122 - val_acc: 0.9714
Epoch 8/20
 - 4s - loss: 0.0244 - acc: 0.9917 - val_loss: 0.1038 - val_acc: 0.9745
Epoch 9/20
 - 4s - loss: 0.0216 - acc: 0.9926 - val_loss: 0.1135 - val_acc: 0.9721
Epoch 10/20
 - 4s - loss: 0.0229 - acc: 0.9927 - val_loss: 0.1309 - val_acc: 0.9670
Epoch 11/20
 - 4s - loss: 0.0185 - acc: 0.9936 - val_loss: 0.1289 - val_acc: 0.9732
Epoch 12/20
 - 4s - loss: 0.0204 - a

In [33]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 5s 153us/step - loss: 0.3397 - acc: 0.9011 - val_loss: 0.1493 - val_acc: 0.9562
Epoch 2/20
33600/33600 [==============================] - 4s 131us/step - loss: 0.1246 - acc: 0.9613 - val_loss: 0.1262 - val_acc: 0.9615
Epoch 3/20
33600/33600 [==============================] - 4s 129us/step - loss: 0.0795 - acc: 0.9754 - val_loss: 0.0992 - val_acc: 0.9707
Epoch 4/20
33600/33600 [==============================] - 4s 129us/step - loss: 0.0551 - acc: 0.9826 - val_loss: 0.0951 - val_acc: 0.9742
Epoch 5/20
33600/33600 [==============================] - 4s 130us/step - loss: 0.0501 - acc: 0.9840 - val_loss: 0.0921 - val_acc: 0.9749
Epoch 6/20
33600/33600 [==============================] - 4s 130us/step - loss: 0.0371 - acc: 0.9884 - val_loss: 0.0899 - val_acc: 0.9751
Epoch 7/20
33600/33600 [==============================] - 4s 134us/step - loss: 0.0242 - acc: 0.9925 - val_loss: 0.1004 - v

In [34]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_5 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total para

In [35]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))



W0806 14:51:46.021346 139844299061120 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

In [36]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [0]:
test_pred.to_csv('mnist_submission.csv', index = False)